# BloomWatch: Model Training Experiments

Training and experimentation notebook for plant bloom detection models.

In [ ]:
# Setup
import sys
import os
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time

project_root = Path(os.getcwd()).parent
sys.path.append(str(project_root))

from data import PlantBloomDataset, create_standard_transforms
from models import SimpleCNN, ResNetBaseline, ModelUtils
from models.losses import FocalLoss
from utils import ConfigManager, MetricsTracker, get_device, set_seed
from visualization import plot_training_metrics

# Set random seed for reproducibility
set_seed(42)
device = get_device()
print(f"Using device: {device}")

In [ ]:
# Load configuration
config_path = project_root / "configs" / "config.yaml"
config_manager = ConfigManager(str(config_path))
config = config_manager.config

print("Training Configuration:")
print(f"Epochs: {config.training.epochs}")
print(f"Batch size: {config.data.batch_size}")
print(f"Learning rate: {config.training.learning_rate}")
print(f"Model: {config.model.name}")

In [ ]:
# Prepare datasets
data_dir = project_root / "data" / "raw"
annotations_file = project_root / "data" / "annotations.csv"

# Create transforms
train_transform = create_standard_transforms(
    image_size=tuple(config.data.image_size),
    is_training=True
)
val_transform = create_standard_transforms(
    image_size=tuple(config.data.image_size),
    is_training=False
)

# Load datasets
try:
    train_dataset = PlantBloomDataset(
        data_dir=str(data_dir),
        annotations_file=str(annotations_file),
        transform=train_transform,
        stage='train'
    )
    
    val_dataset = PlantBloomDataset(
        data_dir=str(data_dir),
        annotations_file=str(annotations_file),
        transform=val_transform,
        stage='val'
    )
    
    print(f"Train dataset: {len(train_dataset)} samples")
    print(f"Val dataset: {len(val_dataset)} samples")
    
except Exception as e:
    print(f"Using dummy datasets: {e}")
    # Create dummy datasets for demonstration
    train_dataset = None
    val_dataset = None

In [ ]:
# Create data loaders
if train_dataset and val_dataset:
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.data.batch_size,
        shuffle=True,
        num_workers=config.data.num_workers
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config.data.batch_size,
        shuffle=False,
        num_workers=config.data.num_workers
    )
    
    print(f"Train batches: {len(train_loader)}")
    print(f"Val batches: {len(val_loader)}")
else:
    print("Skipping data loader creation (no real data)")

In [ ]:
# Initialize model
if config.model.name == 'simple_cnn':
    model = SimpleCNN(
        num_classes=config.model.num_classes,
        dropout=config.model.dropout
    )
elif config.model.name == 'resnet':
    model = ResNetBaseline(
        num_classes=config.model.num_classes,
        backbone=config.model.backbone,
        pretrained=config.model.pretrained,
        dropout=config.model.dropout
    )
else:
    model = SimpleCNN(num_classes=config.model.num_classes)

model = model.to(device)
print(f"Model: {model.__class__.__name__}")

# Count parameters
param_count = ModelUtils.count_parameters(model)
print(f"Parameters: {param_count}")

In [ ]:
# Setup training components
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    model.parameters(),
    lr=config.training.learning_rate,
    weight_decay=config.training.weight_decay
)

scheduler = optim.lr_scheduler.StepLR(
    optimizer,
    step_size=config.training.step_size,
    gamma=config.training.gamma
)

# Metrics tracker
metrics_tracker = MetricsTracker(
    num_classes=config.model.num_classes,
    class_names=['bud', 'early_bloom', 'full_bloom', 'late_bloom', 'dormant']
)

print("Training setup complete!")

In [ ]:
# Training function
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (data, target, _) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)
    
    return total_loss / len(loader), correct / total

def validate_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target, _ in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            
            total_loss += loss.item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)
    
    return total_loss / len(loader), correct / total

In [ ]:
# Training loop
if train_dataset and val_dataset:
    print("Starting training...")
    
    for epoch in range(config.training.epochs):
        start_time = time.time()
        
        # Train
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        
        # Validate
        val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
        
        # Update scheduler
        scheduler.step()
        
        # Track metrics
        metrics = {
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'learning_rate': optimizer.param_groups[0]['lr']
        }
        metrics_tracker.update_epoch_metrics(epoch, metrics)
        
        epoch_time = time.time() - start_time
        
        if epoch % 5 == 0:
            print(f"Epoch {epoch:3d}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
                  f"Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Time: {epoch_time:.2f}s")
    
    print("Training completed!")
else:
    print("Simulating training with dummy data...")
    # Simulate training metrics for demonstration
    for epoch in range(10):
        # Simulate improving metrics
        train_loss = 2.0 - epoch * 0.15 + np.random.normal(0, 0.1)
        val_loss = 2.2 - epoch * 0.12 + np.random.normal(0, 0.1)
        train_acc = 0.2 + epoch * 0.08 + np.random.normal(0, 0.02)
        val_acc = 0.15 + epoch * 0.07 + np.random.normal(0, 0.02)
        
        metrics = {
            'train_loss': max(0.1, train_loss),
            'val_loss': max(0.1, val_loss),
            'train_acc': min(0.95, max(0.1, train_acc)),
            'val_acc': min(0.9, max(0.1, val_acc)),
            'learning_rate': 0.001 * (0.9 ** (epoch // 3))
        }
        metrics_tracker.update_epoch_metrics(epoch, metrics)
    
    print("Simulation completed!")

In [ ]:
# Visualize training results
fig = metrics_tracker.plot_metrics_history()
plt.show()

# Print best metrics
best_metrics = metrics_tracker.best_metrics
print("\nBest Metrics:")
for metric, value in best_metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
# Save model and metrics
save_dir = project_root / "checkpoints"
save_dir.mkdir(exist_ok=True)

# Save model
model_path = save_dir / "best_model.pth"
if train_dataset:  # Only save if we actually trained
    ModelUtils.save_model(
        model=model,
        optimizer=optimizer,
        epoch=best_metrics['best_epoch'],
        loss=best_metrics['best_val_loss'],
        filepath=str(model_path),
        metadata={'config': config, 'metrics': best_metrics}
    )
    print(f"Model saved to {model_path}")

# Save metrics
metrics_path = save_dir / "training_metrics.json"
metrics_tracker.save_metrics(str(metrics_path))
print(f"Metrics saved to {metrics_path}")

print("\nTraining experiment completed!")